# This is an simple enquiry system which provide 5 different kind of plot with selected fields. The plots are as follows;
## 1. Scatter Plot which shows the scatter plot with smooth line by two selected numeric fields.
## 2. Boxplot which shows the boxplot of resale value by select field
## 3. Bar Plot which shows min/average/max resales value for selected town and by selected field
## 4. Pie Chart which shows the top 5 transaction count and resale value by the selected field
## 5. Trend Chart which shows the trend of total transaction count and resale valye by the selected field

In [ ]:
# added town drop down
import tkinter as tk
from tkinter import ttk
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Load the HDB resale dataset
df = pd.read_csv('ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')  # Adjust the path as necessary
import re
# Function to convert 'remaining_lease' to total months
def lease_to_months(lease):
    # Use regex to capture the number of years and months
    years = re.search(r'(\d+)\s+years?', lease)
    months = re.search(r'(\d+)\s+months?', lease)
    
    # Convert to integers, if found, else default to 0 for months if missing
    years = int(years.group(1)) if years else 0
    months = int(months.group(1)) if months else 0

    remain_year = years + round((months/12),2)  # Convert to year
    return remain_year

# Apply the function to the 'remaining_lease' field
df['remaining_lease_year'] = df['remaining_lease'].apply(lease_to_months)
# Get the list of unique towns
towns = df['town'].unique().tolist()
towns.insert(0, "All")  # Add an "All" option to include all towns

# Create the main application window
root = tk.Tk()
root.title("Data Visualization Tool")

# Dropdown options for x-axis, y-axis selections, and y-axis aggregation for bar chart
x_options = ['flat_type', 'flat_model', 'storey_range']  # For boxplot and bar chart
x_options_scatter = ['resale_price', 'floor_area_sqm', 'remaining_lease_year']  # X-axis options for scatter plot
y_options_scatter = ['resale_price', 'floor_area_sqm', 'remaining_lease_year']  # Y-axis options for scatter plot
y_options_bar = ['max', 'min', 'average']  # Aggregation options for bar chart

# Add a title label at the top
title_label = tk.Label(root, text="Data Visualization Tool", font=("Helvetica", 16))
title_label.grid(row=0, column=0, columnspan=2, pady=10)

# Function to update dropdown visibility based on plot type
def update_dropdowns():
    # Forget all widgets first
    x_label.grid_forget()
    x_menu.grid_forget()
    town_label.grid_forget()
    town_menu.grid_forget()
    y_scatter_label.grid_forget()
    y_scatter_menu.grid_forget()
    y_bar_label.grid_forget()
    y_bar_menu.grid_forget()

    # Get selected plot type
    selected_plot = plot_var.get()

    if selected_plot == "Scatter Plot":
        x_label['text'] = "X-axis (Scatter Plot):"
        x_menu['values'] = x_options_scatter
        x_menu.set("Select X-axis")
        x_label.grid(row=2, column=0, sticky='e', padx=5)
        x_menu.grid(row=2, column=1, padx=5)
        
        y_scatter_label.grid(row=3, column=0, sticky='e', padx=5)
        y_scatter_menu.grid(row=3, column=1, padx=5)
        
    elif selected_plot == "Boxplot":
        x_label['text'] = "X-axis (Boxplot):"
        x_menu['values'] = x_options
        x_menu.set("Select X-axis")
        x_label.grid(row=2, column=0, sticky='e', padx=5)
        x_menu.grid(row=2, column=1, padx=5)
        
        town_label.grid(row=3, column=0, sticky='e', padx=5)
        town_menu.grid(row=3, column=1, padx=5)
        
    elif selected_plot == "Bar Chart":
        x_label['text'] = "X-axis (Bar Chart):"
        x_menu['values'] = x_options
        x_menu.set("Select X-axis")
        x_label.grid(row=2, column=0, sticky='e', padx=5)
        x_menu.grid(row=2, column=1, padx=5)
        
        town_label.grid(row=3, column=0, sticky='e', padx=5)
        town_menu.grid(row=3, column=1, padx=5)
        
        y_bar_label.grid(row=4, column=0, sticky='e', padx=5)
        y_bar_menu.grid(row=4, column=1, padx=5)
        
    elif selected_plot == "Pie Chart":
        x_label['text'] = "X-axis (Pie Chart):"
        x_menu['values'] = x_options
        x_menu.set("Select X-axis")
        x_label.grid(row=2, column=0, sticky='e', padx=5)
        x_menu.grid(row=2, column=1, padx=5)
    elif selected_plot == "Trend Chart":
        x_label['text'] = "X-axis (Trend Chart):"
        x_menu['values'] = x_options
        x_menu.set("Select X-axis")
        x_label.grid(row=2, column=0, sticky='e', padx=5)
        x_menu.grid(row=2, column=1, padx=5)

    else:
        tk.messagebox.showinfo("Info", "Please select a valid plot type.")

# Function to plot the scatter plot
def plot_scatter():
    selected_x = x_var.get()
    selected_y = y_scatter_var.get()
    
    if selected_x and selected_y:
        plt.figure(figsize=(10, 6))
        sns.regplot(data=df, x=selected_x, y=selected_y, scatter=True, 
                    scatter_kws={'color': 'skyblue', 'alpha': 0.5}, 
                    line_kws={'color': 'red', 'lw': 2})
        plt.title(f"Scatter Plot of {selected_y.capitalize()} by {selected_x.capitalize()}")
        plt.xlabel(selected_x.capitalize())
        plt.ylabel(selected_y.capitalize())
        plt.xticks(rotation=45)
        
        fig = plt.gcf()
        scatter_window = tk.Toplevel(root)
        scatter_window.title("Scatter Plot")
        canvas = FigureCanvasTkAgg(fig, master=scatter_window)
        canvas.draw()
        canvas.get_tk_widget().pack()


# Function to plot the boxplot
def plot_boxplot():
    selected_x = x_var.get()
    selected_town = town_var.get()
    if selected_x:
        filtered_df = df if selected_town == "All" else df[df['town'] == selected_town]
        plt.figure(figsize=(15, 6))
        filtered_df.boxplot(column='resale_price', by=selected_x)
        plt.title(f'Resale Price by {selected_x.capitalize()} (Town: {selected_town})')
        plt.suptitle("")
        plt.xlabel(selected_x.capitalize())
        plt.ylabel("Resale Price")
        plt.xticks(rotation=45)
        
        fig = plt.gcf()
        boxplot_window = tk.Toplevel(root)
        boxplot_window.title("Boxplot")
        canvas = FigureCanvasTkAgg(fig, master=boxplot_window)
        canvas.draw()
        canvas.get_tk_widget().pack()

# Function to plot the bar chart
def plot_barchart():
    selected_x = x_var.get()
    selected_y_bar = y_bar_var.get()
    selected_town = town_var.get()
    
    if selected_x and selected_y_bar:
        filtered_df = df if selected_town == "All" else df[df['town'] == selected_town]
        if selected_y_bar == 'max':
            data = filtered_df.groupby(selected_x)['resale_price'].max()
        elif selected_y_bar == 'min':
            data = filtered_df.groupby(selected_x)['resale_price'].min()
        elif selected_y_bar == 'average':
            data = filtered_df.groupby(selected_x)['resale_price'].mean()
        
        plt.figure(figsize=(15, 6))
        data.plot(kind='bar', color='skyblue')
        plt.title(f"{selected_y_bar.capitalize()} Resale Price by {selected_x.capitalize()} (Town: {selected_town})")
        plt.xlabel(selected_x.capitalize())
        plt.ylabel("Resale Price")
        plt.xticks(rotation=45)
        
        fig = plt.gcf()
        barchart_window = tk.Toplevel(root)
        barchart_window.title("Bar Chart")
        canvas = FigureCanvasTkAgg(fig, master=barchart_window)
        canvas.draw()
        canvas.get_tk_widget().pack()
        
# Function to plot the pie chart
def plot_piechart():
    selected_x = x_var.get()

    if selected_x:
        grouped_data = df.groupby(selected_x)['resale_price'].count()
        top_5 = grouped_data.nlargest(5)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

        # Pie chart for transaction count
        ax1.pie(top_5, labels=top_5.index, autopct="%1.1f%%", startangle=140)
        ax1.set_title(f"Top 5 Transaction Count by {selected_x.capitalize()}")

        # Pie chart for total resale value
        top_5_value = df.groupby(selected_x)['resale_price'].sum().nlargest(5)
        ax2.pie(top_5_value, labels=top_5_value.index, autopct="%1.1f%%", startangle=140)
        ax2.set_title(f"Top 5 Total Resale Value by {selected_x.capitalize()}")

        fig.suptitle("Pie Chart Analysis")

        pie_window = tk.Toplevel(root)
        pie_window.title("Pie Chart")
        canvas = FigureCanvasTkAgg(fig, master=pie_window)
        canvas.draw()
        canvas.get_tk_widget().pack()

import matplotlib.dates as mdates
# Function to plot the trend chart
def plot_trendchart():
    selected_x = x_var.get()

    if selected_x:
        # Convert 'yyyy-mm' to datetime format
        df['month'] = pd.to_datetime(df['month'], format='%Y-%m')

        # Group data by month and selected group
        grouped_data = df.groupby([pd.Grouper(key='month', freq='M'), selected_x]).size()
        grouped_data = grouped_data.unstack()

        # Calculate total resale value by month and group
        total_value = df.groupby([pd.Grouper(key='month', freq='M'), selected_x])['resale_price'].sum()
        total_value = total_value.unstack()

        # Create the figure and subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Plot transaction count trend
        grouped_data.plot(ax=ax1)
        ax1.set_title(f"Transaction Count Trend by Month and {selected_x.capitalize()}")
        ax1.set_xlabel("Month")
        ax1.set_ylabel("Transaction Count")
        ax1.xaxis.set_major_locator(mdates.MonthLocator())
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %y'))
        ax1.legend(title=selected_x.capitalize())

        # Plot total resale value trend
        total_value.plot(ax=ax2)
        ax2.set_title(f"Total Resale Value Trend by Month and {selected_x.capitalize()}")
        ax2.set_xlabel("Month")
        ax2.set_ylabel("Total Resale Value")
        ax2.xaxis.set_major_locator(mdates.MonthLocator())
        ax2.xaxis.set_major_formatter(mdates.DateFormatter('%b %y'))
        ax2.legend(title=selected_x.capitalize())

        plt.tight_layout()

        trend_window = tk.Toplevel(root)
        trend_window.title("Trend Chart")
        canvas = FigureCanvasTkAgg(fig, master=trend_window)
        canvas.draw()
        canvas.get_tk_widget().pack()
        
# Function to run the selected plot type
def run_plot():
    selected_plot = plot_var.get()
    if selected_plot == "Scatter Plot":
        plot_scatter()
    elif selected_plot == "Boxplot":
        plot_boxplot()
    elif selected_plot == "Bar Chart":
        plot_barchart()
    elif selected_plot == "Pie Chart":
        plot_piechart()
    elif selected_plot == "Trend Chart":
        plot_trendchart()

# Function to exit the application
def exit_app():
    root.quit()
    root.destroy()

# Create a dropdown menu for plot selection with a label in the same row
plot_label = tk.Label(root, text="Option:")
plot_label.grid(row=1, column=0, sticky='e', padx=5)
plot_var = tk.StringVar()
plot_menu = ttk.Combobox(root, textvariable=plot_var, values=["Scatter Plot", "Boxplot", "Bar Chart","Pie Chart", "Trend Chart"], state="readonly")
plot_menu.set("Select Plot Type")
plot_menu.grid(row=1, column=1, padx=5, pady=10)
plot_menu.bind("<<ComboboxSelected>>", lambda e: update_dropdowns())

# Create labels and dropdowns for x-axis, y-axis, and town selections
x_label = tk.Label(root, text="X-axis:")
x_var = tk.StringVar()
x_menu = ttk.Combobox(root, textvariable=x_var, state="readonly")

town_label = tk.Label(root, text="Town:")
town_var = tk.StringVar()
town_menu = ttk.Combobox(root, textvariable=town_var, values=towns, state="readonly")
town_menu.set("Select Town")

y_scatter_label = tk.Label(root, text="Y-axis (for Scatter Plot):")
y_scatter_var = tk.StringVar()
y_scatter_menu = ttk.Combobox(root, textvariable=y_scatter_var, values=y_options_scatter, state="readonly")
y_scatter_menu.set("Select Y-axis (for Scatter Plot)")

y_bar_label = tk.Label(root, text="Y-axis Aggregation (for Bar Chart):")
y_bar_var = tk.StringVar()
y_bar_menu = ttk.Combobox(root, textvariable=y_bar_var, values=y_options_bar, state="readonly")
y_bar_menu.set("Select Y-axis Aggregation (for Bar Chart)")

# Create "Run" and "Exit" buttons
run_button = tk.Button(root, text="Run Plot", command=run_plot)
run_button.grid(row=5, column=0, columnspan=2, pady=10)

exit_button = tk.Button(root, text="Exit", command=exit_app)
exit_button.grid(row=6, column=0, columnspan=2, pady=5)

# Run the application
root.mainloop()


C:\Users\Razer\AppData\Local\Temp\ipykernel_14828\3634528573.py:219: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  grouped_data = df.groupby([pd.Grouper(key='month', freq='M'), selected_x]).size()
C:\Users\Razer\AppData\Local\Temp\ipykernel_14828\3634528573.py:223: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  total_value = df.groupby([pd.Grouper(key='month', freq='M'), selected_x])['resale_price'].sum()
